In [1]:
import os
import re
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import skimage
import anndata as ad
import seaborn as sns
from matplotlib.pyplot import rc_context
from tqdm.notebook import tqdm, trange
from sklearn.preprocessing import MinMaxScaler
sc.settings.verbosity = 3

In [2]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / "data"

In [3]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
import scSpaMet as spamet

In [5]:
experiment = "endometrium"
cores = ["L128"]
datasets = ["L11_2", "L11_3"]+[core + f"_{i}" for core in cores for i in range(1, 4)]

In [6]:
import palettable
heatmap_cmp = palettable.cmocean.diverging.Balance_20.mpl_colormap


# Load data

In [7]:
def read_props(dataset, experiment):
    df_morph = pd.read_csv(data_dir / "props" / f"morphology_IMC_{experiment}_{dataset}.csv")
    df_intensity_IMC = pd.read_csv(data_dir / "props" / f"intensity_IMC_{experiment}_{dataset}.csv")
    df_intensity_TS = pd.read_csv(data_dir / "props" / f"intensity_TS_{experiment}_{dataset}_auto.csv")

    return df_morph, df_intensity_IMC, df_intensity_TS

In [8]:
%%capture 

adatas = []
adatas_raw = []
df_sums = []
for dataset in datasets:
    try:
        df_morph, df_intensity_IMC, df_intensity_TS = read_props(dataset, experiment)
        print(f'Sucessfully read dataset {dataset}') 
    except:
        print(f'{dataset} cannot be read') 
        continue
    try:
        df_intensity_TS.drop(['Rest', 'Total'], axis=1, inplace=True)
    except:
        pass
    df_intensity_TS.iloc[:, 1:] = df_intensity_TS.iloc[:, 1:].multiply(
        df_morph["area"], axis=0
    )
    # data_all = df_intensity_TS.iloc[:, 1:].values
    # data_all_norm = (data_all+0.1)/(np.percentile(data_all,50,axis=1,keepdims=True)+0.1)
    # data_all_norm = MinMaxScaler().fit_transform(data_all)
    # df_intensity_TS.iloc[:, 1:] = data_all_norm
    
    # Merge TS and IMC data
    df = df_intensity_TS
    df.set_index("Id", inplace=True)
    df_sums.append(df.sum(axis=0))
    
    # Put to adata format
    adata = sc.AnnData(df.values)
    adata.var_names = df.columns.tolist()
    adata.obs["Cell"] = df.index.tolist()
    adata.obs["Dataset"] = dataset
    adata.obs["Core"] = dataset.split("_")[0]
    adata.obsm["spatial"] = df_morph[["centroid-0", "centroid-1"]].to_numpy()
    
    # FPM normalize
    sc.pp.normalize_total(adata, target_sum=1e5)
    # sc.pp.log1p(adata, base=2)
    adatas_raw.append(adata.copy())
    
    # Standard scale
    sc.pp.scale(adata)
    adatas.append(adata)
    
adata = ad.concat(adatas, join="inner")
adata_raw = ad.concat(adatas_raw, join="inner")

normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)


In [9]:
# Read Protein clustering info
path = data_dir / "adata" / f"{experiment}.h5ad"
adata_IMC_cluster = ad.read_h5ad(path)


C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [10]:
adata = adata[~adata.obs.Dataset.isin(['F7_1', 'F7_2'])]
adata_IMC = adata_IMC_cluster[~adata_IMC_cluster.obs.Dataset.isin(['F7_1', 'F7_2'])]

In [11]:
df_sum = pd.concat(df_sums, axis=1).mean(axis=1)
# mz_qc = df_sum[(df_sum > 16000)].index.tolist()
mz_qc = df_sum[(df_sum > 6000)].index.tolist()

In [12]:
df_sum 

1.0        1402.456701
12.0       2735.992446
13.0      12847.610014
14.0     343037.893689
15.0     629932.177484
             ...      
221.9      6417.500514
228.9     16076.675840
233.9      7226.838879
237.9      5124.135022
242.9      3631.294195
Length: 192, dtype: float64

In [13]:
len(mz_qc)

184

In [14]:
adata = adata[:, mz_qc]
adata_raw = adata_raw[:, mz_qc]

In [15]:
# Define IMC and SIMS marker list
IMC_markers = adata.var_names.tolist()
SIMS_masses = adata_IMC.var_names.tolist()

In [24]:
SIMS_masses

['CD3', 'CD8a', 'Ecaderin', 'PanKeratin', 'SMA', 'Vimentin']

# Joint embedding

In [16]:
def Umap(a,rep=None):
    if rep is not None:
        print(f'Using representation {rep}')
        sc.pp.neighbors(a,use_rep=rep, metric='cosine')
    else:
        sc.pp.neighbors(a)
    sc.tl.umap(a)
    print('Sucessfully run Umap!')
    return a

def TSNE(a,rep=None):
    if rep is not None:
        print(f'Using representation {rep}')
        sc.tl.tsne(a,use_rep=rep)
    else:
        sc.tl.tsne(a)
    print('Sucessfully run TSNE!')
    return a

In [17]:
train_x_IMC = adata_IMC.X
train_x_SIMS = adata.X

In [18]:
# X_embedding, _, _ = spamet.tl.Joint_XVAE_clustering(train_x_IMC, train_x_SIMS, epochs=20)
X_embedding, q, X_predict = spamet.tl.Joint_XVAE_clustering(train_x_IMC, train_x_SIMS, epochs=50, resolution=0.3)

GPU name:  []


C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/50
257/257 - 4s - loss: 0.7170 - reconstruction_loss: 0.7021 - kl_loss: 1.4910 - 4s/epoch - 17ms/step
Epoch 2/50
257/257 - 1s - loss: 0.5641 - reconstruction_loss: 0.5554 - kl_loss: 0.8777 - 621ms/epoch - 2ms/step
Epoch 3/50
257/257 - 1s - loss: 0.5253 - reconstruction_loss: 0.5195 - kl_loss: 0.5779 - 604ms/epoch - 2ms/step
Epoch 4/50
257/257 - 1s - loss: 0.5027 - reconstruction_loss: 0.4974 - kl_loss: 0.5255 - 597ms/epoch - 2ms/step
Epoch 5/50
257/257 - 1s - loss: 0.4839 - reconstruction_loss: 0.4783 - kl_loss: 0.5568 - 628ms/epoch - 2ms/step
Epoch 6/50
257/257 - 1s - loss: 0.4731 - reconstruction_loss: 0.4674 - kl_loss: 0.5666 - 605ms/epoch - 2ms/step
Epoch 7/50
257/257 - 1s - loss: 0.4664 - reconstruction_loss: 0.4607 - kl_loss: 0.5749 - 616ms/epoch - 2ms/step
Epoch 8/50
257/257 - 1s - loss: 0.4612 - reconstruction_loss: 0.4554 - kl_loss: 0.5753 - 624ms/epoch - 2ms/step
Epoch 9/50
257/257 - 1s - loss: 0.4572 - reconstruction_loss: 0.4514 - kl_loss: 0.5834 - 686ms/epoch - 3ms

C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\umap\distances.py:1086: NumbaDepr

    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:34)
running Leiden clustering
    finished: found 7 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:01)
The value of delta_label of current 1 th iteration is 0.18807060255629945 >= tol 0.005
Epoch 1/50
257/257 [==============================] - 3s 2ms/step - loss: 0.0784
Epoch 2/50
257/257 [==============================] - 0s 2ms/step - loss: 0.0365
Epoch 3/50
257/257 [==============================] - 0s 2ms/step - loss: 0.0261
Epoch 4/50
257/257 [==============================] - 0s 2ms/step - loss: 0.0229
Epoch 5/50
257/257 [==============================] - 0s 2ms/step - loss: 0.0213
Epoch 6/50
257/257 [==============================] - 0s 2ms/step - loss: 0.0194
Epoch 7/50
257/257 [==============================] - 0s 2ms/step - loss: 0.0187
Epoch 8/50
257/257 [==================

In [19]:
_, X_predict_IMC = spamet.tl.Joint_VAE(train_x_IMC, epochs=50,  netwidths=[16,8], latent_dim=8)

_, X_predict_SIMS = spamet.tl.Joint_VAE(train_x_SIMS, epochs=50,  netwidths=[128,64,32,16], latent_dim=8)

GPU name:  []


C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/50
257/257 - 3s - loss: 0.2221 - reconstruction_loss: 0.2060 - kl_loss: 0.1615 - 3s/epoch - 12ms/step
Epoch 2/50
257/257 - 0s - loss: 0.1081 - reconstruction_loss: 0.0994 - kl_loss: 0.0867 - 356ms/epoch - 1ms/step
Epoch 3/50
257/257 - 0s - loss: 0.0894 - reconstruction_loss: 0.0833 - kl_loss: 0.0615 - 367ms/epoch - 1ms/step
Epoch 4/50
257/257 - 0s - loss: 0.0738 - reconstruction_loss: 0.0656 - kl_loss: 0.0816 - 363ms/epoch - 1ms/step
Epoch 5/50
257/257 - 0s - loss: 0.0630 - reconstruction_loss: 0.0543 - kl_loss: 0.0870 - 360ms/epoch - 1ms/step
Epoch 6/50
257/257 - 0s - loss: 0.0553 - reconstruction_loss: 0.0452 - kl_loss: 0.1008 - 371ms/epoch - 1ms/step
Epoch 7/50
257/257 - 0s - loss: 0.0474 - reconstruction_loss: 0.0363 - kl_loss: 0.1109 - 370ms/epoch - 1ms/step
Epoch 8/50
257/257 - 0s - loss: 0.0440 - reconstruction_loss: 0.0328 - kl_loss: 0.1119 - 390ms/epoch - 2ms/step
Epoch 9/50
257/257 - 0s - loss: 0.0416 - reconstruction_loss: 0.0301 - kl_loss: 0.1150 - 401ms/epoch - 2ms

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

r_IMC = mean_absolute_error(train_x_IMC, X_predict[0])
r_SIMS = mean_absolute_error(train_x_SIMS, X_predict[1])
print(r_IMC, r_SIMS)

0.10931712 0.6200699976707201


In [21]:
r_IMC = mean_absolute_error(train_x_IMC, X_predict_IMC)
r_SIMS = mean_absolute_error(train_x_SIMS, X_predict_SIMS)
print(r_IMC, r_SIMS)

0.10189014 0.6306865182174954
